In this notebook we want to assess the difference in noise correlations between models trained with and without dropout.

We do that by:

1. Training the networks with and without dropouts
2. Collecting the embedding layers activations on test data
3. Training classifiers on the embedding layers suffled activity, and assessing it unshuffled test data (for both dropout and non-dropout networks)
4. Assessing covariance of embedding layer 

5. Finally we vary the dropout probability and determine noise correlations as a function of dropout probability (or batch size, not sure)

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


We make the most vanilla convnet to run MNIST. This is literally the exact conv-net from the 'deep mnist for experts' tutorial. 

We can specify at training time the keep probability, which when we want to set a network with no dropout we just set to 0. 

In [ ]:
sess = tf.Session()

In [4]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [5]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [6]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())

In [8]:
for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(session = sess, feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(session = sess, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(session = sess, feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.18
step 100, training accuracy 0.88
step 200, training accuracy 0.94
step 300, training accuracy 0.96
step 400, training accuracy 0.92
step 500, training accuracy 0.98
step 600, training accuracy 0.92
step 700, training accuracy 0.94
step 800, training accuracy 0.96
step 900, training accuracy 0.96
step 1000, training accuracy 1
step 1100, training accuracy 0.98
step 1200, training accuracy 0.96
step 1300, training accuracy 0.94
step 1400, training accuracy 0.96
step 1500, training accuracy 0.94
step 1600, training accuracy 1
step 1700, training accuracy 1
step 1800, training accuracy 0.98
step 1900, training accuracy 0.96


ResourceExhaustedError: OOM when allocating tensor with shape[10000,28,28,32]
	 [[Node: Conv2D = Conv2D[T=DT_FLOAT, padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/cpu:0"](Reshape, Variable/read)]]
Caused by op u'Conv2D', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/

In [56]:
#trials = [sess.run(h_fc1_drop, feed_dict = {x: mnist.test.images[0:100], keep_prob: .5}) for i in range(50)]
#on a small amazon instance can't do more than 10000 training examples. Will fix this tmrw once the code is written. 

all_units = sess.run(h_fc1_drop, feed_dict = {x: mnist.test.images[0:1000], keep_prob: 1})

In [57]:
import numpy as np

ylabels = [np.argmax(row) for row in mnist.test.labels[0:1000]] 

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder

#ylabels = LabelEncoder().fit_transform(mnist.test.labels[0:1000])

X_train, X_test, y_train, y_test = train_test_split(all_units, ylabels)

model = LogisticRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.96399999999999997

In [128]:
def tensorize(data, labels, est_trials = 100, shuffle = False):
    n_points, n_neurons = data.shape
    n_classes = len(set(labels))
    
    tensor = np.empty([n_classes, est_trials, n_neurons])
    trial_count = np.zeros(n_classes).astype('int')
    
    for i in range(n_points):
        
        tensor[labels[i]][trial_count[labels[i]]] = data[i]
        trial_count[labels[i]] += 1
        
    num_trials = min(trial_count).astype('int')
    tensor = tensor[:, 0:num_trials,: ]
    
    if shuffle:
        
        for i in range(n_neurons):
            for j in range(n_classes):      
                tensor[j, :, i] = np.random.permutation(tensor[j, :, i])

    labels = [range(10) for i in range(num_trials)]
    labels = np.array(train_labels).transpose()
    
    return tensor, labels

In [132]:
train_tensor, train_labels = tensorize(X_train, y_train, shuffle = True)
test_tensor, test_labels = tensorize(X_test, y_test, shuffle = False)

train_tensor, train_labels = train_tensor.reshape(-1, 1024), train_labels.reshape([-1])
test_tensor, test_labels = test_tensor.reshape(-1, 1024), test_labels.reshape([-1])

In [133]:
model = LogisticRegression()
model.fit(train_tensor, train_labels)
model.score(X_test, y_test)

0.93600000000000005